In [ ]:
import time
import pandas as pd
import numpy as np
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Custom print function for timing information
def print_timer_info(message):
    print(f"[TIMER INFO] {message}")

# Automatically download missing NLTK resources without print messages
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Check CUDA availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: {}'.format(device))

# Load the dataset
start_time = time.time()
print_timer_info("Loading the 20 Newsgroups dataset...")
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({'Text': newsgroups.data, 'Category': newsgroups.target})
df['Category Name'] = df['Category'].apply(lambda x: newsgroups.target_names[x])
print_timer_info(f"Time taken to load dataset: {time.time() - start_time:.2f} seconds")




# Pre-processing
print_timer_info("Starting pre-processing...")
preprocess_start_time = time.time()

df['Text'] = df['Text'].str.lower()  # Lowercasing
df['Text'] = df['Text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))  # Remove punctuation/special characters
df['Tokens'] = df['Text'].apply(word_tokenize)  # Tokenization

stop_words = set(stopwords.words('english'))
df['Tokens'] = df['Tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])  # Stopwords removal

stemmer = PorterStemmer()
df['Tokens'] = df['Tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])  # Stemming

print_timer_info(f"Total time for pre-processing: {time.time() - preprocess_start_time:.2f} seconds")

# Data Preparation
data_prep_start_time = time.time()
print_timer_info("Starting data preparation...")

# Define and fit the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=6620, stop_words='english')  # Set max_features to limit vocab size if necessary
X_tfidf = tfidf_vectorizer.fit_transform(df['Text']).toarray()  # Assuming you are using the original 'Text' column

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Category'])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create PyTorch Dataset
class TfidfDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TfidfDataset(X_train_tensor, y_train_tensor)
val_dataset = TfidfDataset(X_val_tensor, y_val_tensor)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print_timer_info(f"Total time for data preparation: {time.time() - data_prep_start_time:.2f} seconds")

# Define the CNN-LSTM model
class CNNLSTMClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, num_lstm_layers=1, kernel_size=3, num_filters=100):
        super(CNNLSTMClassifier, self).__init__()
        
        # # CNN layer
        # self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=kernel_size)
        # self.pool = nn.MaxPool1d(kernel_size=2)
        # 
        # # LSTM layer
        # self.lstm = nn.LSTM(input_size=num_filters, hidden_size=hidden_dim, num_layers=num_lstm_layers, batch_first=True)
        # 
        # # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Reshape input for Conv1d (batch_size, input_dim, sequence_length)
        # x = x.unsqueeze(1)  # Add a channel dimension (batch_size, 1, input_dim)
        # 
        # # CNN operations
        # x = self.conv1(x)
        # x = F.relu(x)
        # x = self.pool(x)
        # 
        # # Reshape for LSTM (batch_size, sequence_length, num_filters)
        # x = x.permute(0, 2, 1)
        # 
        # # LSTM operations
        # x, _ = self.lstm(x)
        # 
        # # Take the last hidden state
        # x = x[:, -1, :]  # (batch_size, hidden_dim)
        # 
        # # Fully connected layer
        x = self.fc(x)
        
        return x

# Model Parameters
input_dim = X_train.shape[1]  # Ensure it matches the number of features
output_dim = len(label_encoder.classes_)

# Instantiate the new model
model = CNNLSTMClassifier(input_dim=input_dim, output_dim=output_dim)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Move model to GPU if available
model = model.to(device)
criterion = criterion.to(device)

# Training the model with early stopping
num_epochs = 50
patience = 5
best_val_loss = float('inf')
epochs_no_improve = 0

print_timer_info("Starting model training...")
training_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    epoch_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        predictions = model(X_batch)

        # Calculate loss
        loss = criterion(predictions, y_batch)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate epoch loss
        epoch_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted_classes = torch.max(predictions, 1)
            correct_predictions += (predicted_classes == y_batch).sum().item()
            total_predictions += y_batch.size(0)

    val_acc = correct_predictions / total_predictions
    avg_val_loss = val_loss / len(val_loader)

    # Print the results for the current epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss / len(train_loader):.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_acc:.4f}, '
          f'Time Taken: {time.time() - epoch_start_time:.2f} seconds')

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        # Save the best model state
        best_model_state = model.state_dict()
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f'Early stopping triggered after {epoch + 1} epochs.')
        break

print_timer_info(f"Total training time: {time.time() - training_start_time:.2f} seconds")

# Load the best model state if needed
model.load_state_dict(best_model_state)

# Model Evaluation
eval_start_time = time.time()
print_timer_info("Starting model evaluation...")

# Switch model to evaluation mode
model.eval()

# Initialize lists to store the predictions and true labels
all_predictions = []
all_labels = []

# Disable gradient computation for evaluation
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Get model predictions
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)

        # Store predictions and labels
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# Convert to NumPy arrays for evaluation
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

# Print evaluation results
print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print_timer_info(f"Total evaluation time: {time.time() - eval_start_time:.2f} seconds")



print_timer_info("Generating classification report...")
report_start_time = time.time()

# Generate a classification report
report = classification_report(all_labels, all_predictions, target_names=list(map(str, label_encoder.classes_)))
print(report)

print_timer_info(f"Time taken to generate classification report: {time.time() - report_start_time:.2f} seconds")